# Latent Attacks with Normalizing Flows

* We will use [FrEIA](https://github.com/VLL-HD/FrEIA) framework written in [PyTorch](https://pytorch.org/) to easily implement normalizing flows.


In [ ]:
# install FrEIA framework
!pip install git+https://github.com/VLL-HD/FrEIA.git

In [ ]:
# global imports
import random

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

import FrEIA.framework as Ff
import FrEIA.modules as Fm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATASET = 'mnist' # 'fmnist'
MODEL = 'lenet' # 'standard'
BATCH_SIZE = 100

# reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

## Models

Let's start by defining our normalizing flows. Here is a normalizing flows directly taken from [FrEIA](https://github.com/VLL-HD/FrEIA) main page

In [ ]:
def subnet_fc(c_in, c_out, c_hidden=512):
    return nn.Sequential(nn.Linear(c_in, c_hidden), nn.ReLU(),
                         nn.Linear(c_hidden, c_out))

def subnet_conv(c_in, c_out, c_hidden=256):
    return nn.Sequential(nn.Conv2d(c_in, c_hidden, 3, padding=1), nn.ReLU(),
                         nn.Conv2d(c_hidden, c_out, 3, padding=1))

def subnet_conv_1x1(c_in, c_out, c_hidden=256):
    return nn.Sequential(nn.Conv2d(c_in, c_hidden, 1), nn.ReLU(),
                         nn.Conv2d(c_hidden, c_out, 1))

def build_flow(layers=12):
    cond = Ff.ConditionNode(10, name='condition')
    nodes = [Ff.InputNode(28*28, name='input')]

    for k in range(layers):
        nodes.append(Ff.Node(nodes[-1],
                            Fm.GLOWCouplingBlock,
                            {'subnet_constructor': subnet_fc, 'clamp':2.0},
                            conditions=cond,
                            name=F'coupling_{k}'))
        nodes.append(Ff.Node(nodes[-1],
                            Fm.PermuteRandom,
                            {'seed':k},
                            name=F'permute_{k}'))

    nodes.append(Ff.OutputNode(nodes[-1], name='output'))
    return Ff.ReversibleGraphNet(nodes + [cond])

Next, we will define simple convolution classfiers.

In [ ]:
from scipy.stats import truncnorm

def truncated_normal(size, threshold=1):
    """Samples values from truncated normal distribution centered at 0
    Args:
        size: shape or amount of samples
        threshold: cut-off value for distribution
    Returns:
        numpy array of given size
    """
    return truncnorm.rvs(-threshold, threshold, size=size)


class LeNet(nn.Module):
    """LeNet-5 from `"Gradient-Based Learning Applied To Document Recognition"
    <http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf>`_
    """

    def __init__(self, num_classes=10) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        self._init_weights()

    # ref: https://discuss.pytorch.org/t/implementing-truncated-normal-initializer/4778
    def _init_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                # truncated normal distribution with std 0.1 (truncate > 2 x std)
                # https://www.tensorflow.org/api_docs/python/tf/random/truncated_normal
                weights = truncated_normal(list(m.weight.shape), threshold=0.1 * 2)
                weights = torch.from_numpy(weights)
                m.weight.data.copy_(weights)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.avgpool(self.relu(self.conv1(x)))
        x = self.avgpool(self.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
 class Classifier(torch.nn.Module):
    """
    Simple classifier.
    """

    def add_layer(self, name, layer):
        """
        Add a layer.
        :param name:
        :param layer:
        :return:
        """

        setattr(self, name, layer)
        self.layers.append(name)
    
    def __init__(self, N_class=10, resolution=[1,28,28]):
        super(Classifier, self).__init__()
        self.layers = []

        layer = 0
        channels = []
        resolutions = []

        # Determine parameters of network
        activation = torch.nn.ReLU
        gain = 1
        if activation:
            gain = torch.nn.init.calculate_gain('relu')

        batch_normalization = True
        dropout = False
        start_channels = 16
        kernel_size = 4
        assert kernel_size%2 == 0

        while True:
            input_channels = resolution[0] if layer == 0 else channels[layer - 1]
            output_channels = start_channels if layer == 0 else channels[layer - 1]*2

            conv = torch.nn.Conv2d(input_channels, output_channels, kernel_size=kernel_size, stride=2, padding=kernel_size//2-1)
            torch.nn.init.kaiming_normal_(conv.weight, gain)
            torch.nn.init.constant_(conv.bias, 0)
            self.add_layer('conv%d' % layer, conv)

            if batch_normalization:
                bn = torch.nn.BatchNorm2d(output_channels)
                torch.nn.init.constant_(bn.weight, 1)
                torch.nn.init.constant_(bn.bias, 0)
                self.add_layer('bn%d' % layer, bn)

            if activation:
                relu = activation(True)
                self.add_layer('act%d' % layer, relu)

            channels.append(output_channels)
            resolutions.append([
                resolution[1] // 2 if layer == 0 else resolutions[layer - 1][0] // 2,
                resolution[2] // 2 if layer == 0 else resolutions[layer - 1][1] // 2
            ])
            if resolutions[-1][0] // 2 < 2 or resolutions[-1][1] < 2:
                break;

            layer += 1

        representation = int(resolutions[-1][0]*resolutions[-1][1]*channels[-1])
        view = Lambda(lambda x: x.view(x.size(0), -1))
        self.add_layer('view', view)

        linear1 = torch.nn.Linear(representation, 10*N_class)
        linear2 = torch.nn.Linear(10*N_class, N_class)

        torch.nn.init.kaiming_normal_(linear1.weight, gain)
        #torch.nn.init.normal_(linear1.weight, 0, 0.001)
        torch.nn.init.constant_(linear1.bias, 0)
        torch.nn.init.kaiming_normal_(linear2.weight, gain)
        #torch.nn.init.normal_(linear2.weight, 0, 0.001)
        torch.nn.init.constant_(linear2.bias, 0)

        self.add_layer('representation', linear1)
        if dropout:
            drop = torch.nn.Dropout()
            self.add_layer('drop%d' % layer, drop)
        self.add_layer('logits', linear2) 
 
    def forward(self, image, return_features=False):
        """
        Forward pass, takes an image and outputs the predictions.
        :param image: input image
        :type image: torch.autograd.Variable
        :param return_representation: whether to also return representation layer
        :type return_representation: bool
        :return: logits
        :rtype: torch.autograd.Variable
        """

        features = []
        output = image

        for name in self.layers:
            output = getattr(self, name)(output)
            features.append(output)
        if return_features:
            return output, features
        else:
            return output

    def __str__(self):
        """
        Print network.
        """

        string = ''
        for name in self.layers:
            string += '(' + name + ', ' + getattr(self, name).__class__.__name__ + ')\n'
            if type(getattr(self, name)).__class__.__name__ == 'Sequential':
                for module in getattr(self, name).modules():
                    string += '\t(' + module.__class__.__name__ + ')'
        return string

In [ ]:
#define a custom layer for view operations
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

def build_conv(net: str = MODEL):
    if net == 'lenet':
        return LeNet()
    elif net == 'standard':
        return Classifier()
    else:
        raise ValueError

## Dataset

In [ ]:
import collections
from typing import Callable, List, Optional, Tuple
from torch.utils.data import DataLoader, Dataset, Subset, random_split

def split_dataset(dataset: Dataset, split: float, seed: int) -> Tuple[Subset, Subset]:
    """Splits dataset into a train / val set based on a split value and seed

    Args:
        dataset: dataset to split
        split: The proportion of the dataset to include in the validation split,
            must be between 0 and 1.
        seed: Seed used to generate the split

    Returns:
        Subsets of the input dataset

    """
    # Verify that the dataset is Sized
    if not isinstance(dataset, collections.abc.Sized):
        raise ValueError("Dataset is not Sized!")

    if not (0 <= split <= 1):
        raise ValueError(f"Split value must be between 0 and 1. Value: {split}")

    val_length = int(len(dataset) * split)
    train_length = len(dataset) - val_length
    splits = random_split(
        dataset,
        [train_length, val_length],
        generator=torch.Generator().manual_seed(seed),
    )
    return splits


In [ ]:
from typing import Tuple

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

def prepare_dataset(
    dataset: str = DATASET,
    augmentation: bool = False,
    split: float = 0,
    seed: int = 9999,
    batch_size = BATCH_SIZE) -> Tuple[DataLoader, DataLoader]:
    train_augmentations = [transforms.ToTensor()]
    if augmentation:
        train_augmentations += [
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(0.15),
            transforms.RandomCrop(28, padding=4)
        ]

    # download data and prepare loaders

    if dataset == 'mnist':
        train_set = datasets.MNIST(
            'mnist/train',
            train=True,
            download=True,
            transform=transforms.Compose(
                train_augmentations
            ))

        test_set = datasets.MNIST(
            'mnist/train',
            train=False,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ]))
    elif dataset == 'fmnist':
        train_set = datasets.FashionMNIST(
            'fmnist/train',
            train=True,
            download=True,
            transform=transforms.Compose(
                train_augmentations
            ))

        test_set = datasets.FashionMNIST(
            'fmnist/train',
            train=False,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ]))
    else:
        raise ValueError

    if split > 0:
        train_set, _ = split_dataset(
            dataset=train_set, split=split, seed=seed,
        )

    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True)

    test_loader = DataLoader(
        test_set,
        batch_size=batch_size,
        shuffle=False)
    
    return train_loader, test_loader

## Training

We will train various models:

1. Normalizing Flow for generation of images
2. Convolutional Classifier on standard images
3. Convolutional Classifier on Flow outputs
4. Convolutional Classifier on Flow-perturbed images


### Generative Flow

Let's train our conditional flow with MNIST/FashionMNIST.

In [ ]:
import time
from tqdm import tqdm
from torch.optim import Adam

# training function for flow
def train_flow(flow):
    # define hyperparams
    ### optimizer
    lr = 1e-6
    betas = (0.9, 0.999)
    eps = 1e-8
    weight_decay = 0

    ### training
    epochs = 100
    noise = 0.1

    # define optimizer
    flow_params = list(filter(lambda p: p.requires_grad, flow.parameters()))
    flow_optim = Adam(
        flow_params,
        lr=lr,
        betas=betas,
        eps=eps,
        weight_decay=weight_decay)

    # preprocess data for flow
    def prepare_inputs(x, y):
        # prepare shapes
        x = x.view(-1, 1*28*28)
        y = F.one_hot(y, num_classes=10)

        # add to cuda
        x, y = x.cuda(), y.cuda()

        # add a small noise to inputs
        x += noise * torch.cuda.FloatTensor(x.shape).normal_()
        
        # clamp if x is out of bounds
        x = torch.clamp(x, 0., 1.)

        # return as a tuple
        return x, y

    # calculate loss of flow model
    def flow_loss(x, y):
        # feed-forward
        z, log_jac_det = flow(x, y)

        # calculate prior (standard Gaussian) & jacobian density
        prior = torch.sum(z**2, dim=1)
        neg_log = 0.5 * prior - log_jac_det
        
        # calculate loss
        return torch.mean(neg_log)

    # TODO: Measure training time!
    metrics = { 'train_loss': [], 'eval_loss': [] }
    for epoch in range(epochs):
        ## define logger
        #pbar = tqdm(
        #    desc=f'Epoch {epoch+1}/{epochs}',
        #    total=len(train_loader),
        #    position=0,
        #    leave=True)
        
        # start training
        flow.train()
        train_loss = []
        for i, (x, y) in enumerate(train_loader):
            # prepare inputs
            x, y = prepare_inputs(x, y)
            
            # calculate loss
            loss = flow_loss(x, y)
            train_loss.append(loss.cpu().detach())

            # optimization step
            flow_optim.zero_grad()
            loss.backward()
            flow_optim.step()

            ## update bar
            #pbar.update(1)
            #pbar.set_description(f'Epoch: {epoch+1}/{epochs}, Loss: {loss.cpu()}')
        
        # store and print training loss
        metrics['train_loss'].append(np.mean(np.array(train_loss)))
        print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {metrics["train_loss"][-1]}')

        # start evaluation
        flow.eval()
        with torch.no_grad():
            losses = []
            for (x, y) in test_loader:
                # prepare inputs
                x, y = prepare_inputs(x, y)

                # calculate loss
                loss = flow_loss(x, y)

                # save loss
                losses.append(loss.cpu().detach())
            
            # calc mean validation loss
            loss = np.mean(np.array(losses))

            # store and print validation loss
            metrics['eval_loss'].append(loss)
            print(f'Epoch: {epoch+1}/{epochs}, Val Loss: {metrics["eval_loss"][-1]}')
    return flow, flow_optim, metrics

set_seed(9999)
train_loader, test_loader = prepare_dataset()

# define model
print('Preparing generative flow... \n\n')
flow = build_flow()
flow = flow.cuda()

# train the model
flow, flow_optim, flow_metrics = train_flow(flow)

Test generation capabilities of flow.

In [ ]:
# temp for sampling
temp = 0.3

# sample images
with torch.no_grad():
    cond = torch.randint(low=0, high=10, size=(256,))
    cond = F.one_hot(cond, num_classes=10)
    cond = cond.cuda()

    z = temp * torch.cuda.FloatTensor(256, 1*28*28).normal_()
    images, _ = flow(z, cond, rev=True)
    images = images.view(-1, 28, 28)

# take images to cpu for viz
cond = cond.cpu()    
images = images.cpu()

# clamp images
if True:
    images = torch.clamp(images, 0., 1.)

f, axs = plt.subplots(4, 4, figsize=(20,20))
for i in range(4):
    for j in range(4):
        index = i*4+j
        im = axs[i][j].imshow(images[index], cmap='gray')
        axs[i][j].set_title(f'Condition: {np.argmax(cond[index])}', fontsize=20)
        f.colorbar(im, ax=axs[i][j])

Save the model with optimizer and plot training and eval curves.

In [ ]:
def save_model(model, optim, metrics, path=None):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'metrics': metrics,
    }, path)

def plot_flow(metrics):
    plt.figure(figsize=(12,6))
    plt.plot(metrics['train_loss'], label='Training', linewidth=3)
    plt.plot(metrics['eval_loss'], label='Test', linewidth=3)
    plt.title('Flow', fontsize=25)
    plt.xlabel('Epochs', fontsize=25)
    plt.ylabel('NLL', fontsize=25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20)

# save the model
save_model(flow, flow_optim, flow_metrics, path='flow.model')

# plot training curve
plot_flow(flow_metrics)

### Convolutional Classifiers

First write an abstract training function for different settings we have.

In [ ]:
from tqdm import tqdm
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

def train_conv(conv, preprocess_fn):
    # define hyperparams
    ### optimizer
    lr = 0.01
    betas = (0.9, 0.999)
    eps = 1e-8
    weight_decay = 1e-4 #1e-5

    ### training
    epochs = 20

    # define optimizer & loss
    conv_params = list(filter(lambda p: p.requires_grad, conv.parameters()))
    conv_optim = Adam(
        conv_params,
        lr=lr,
        betas=betas,
        eps=eps,
        weight_decay=weight_decay)
    conv_sched = ExponentialLR(
        conv_optim,
        gamma=0.9
    )
    conv_loss = nn.CrossEntropyLoss()

    # TODO: Measure training time!
    metrics = { 'train_loss': [], 'train_acc': [], 'eval_loss': [], 'eval_acc': [] }
    for epoch in range(epochs):
        ## define logger
        #pbar = tqdm(
        #    desc=f'Epoch {epoch+1}/{epochs}',
        #    total=len(train_loader),
        #    position=0,
        #    leave=True)
        
        # start training
        train_loss = []
        train_acc = []
        for (x, y) in train_loader:
            conv.train()

            # preprocess data
            x, y = preprocess_fn(x, y)

            # feed-forward
            preds = conv(x)
            
            # calculate loss & acc
            loss = conv_loss(preds, y)
            acc = torch.mean((torch.argmax(preds, axis=-1) == y) * 1.0)

            # save loss & acc
            train_loss.append(loss.cpu().detach())
            train_acc.append(acc.cpu().detach())

            # optimization step
            conv_optim.zero_grad()
            loss.backward()
            conv_optim.step()

            ## update bar
            #pbar.update(1)
            #pbar.set_description(f'Epoch: {epoch+1}/{epochs}, Loss: {loss.cpu()}')
        
        conv_sched.step()

        # store and print training loss
        metrics['train_loss'].append(np.mean(np.array(train_loss)))
        metrics['train_acc'].append(np.mean(np.array(train_acc)))
        print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {metrics["train_loss"][-1]}, Training Acc: {metrics["train_acc"][-1]}')

        # start evaluation
        conv.eval()
        with torch.no_grad():
            losses = []
            accs = []
            for (x, y) in test_loader:
                # put data on cuda (always test on real test set)
                x, y = x.cuda(), y.cuda()

                # feed-forward
                preds = conv(x)

                # calculate loss & acc
                loss = conv_loss(preds, y)
                acc = torch.mean((torch.argmax(preds, axis=-1) == y) * 1.0)

                # save loss & acc
                losses.append(loss.cpu().detach())
                accs.append(acc.cpu().detach())
            
            # calc mean validation loss
            loss = np.mean(np.array(losses))
            acc = np.mean(np.array(accs))
            
            # store and print validation loss
            metrics['eval_loss'].append(loss)
            metrics['eval_acc'].append(acc)
            print(f'Epoch: {epoch+1}/{epochs}, Val Loss: {metrics["eval_loss"][-1]}, Val Acc: {metrics["eval_acc"][-1]}')

    return conv, conv_optim, metrics

In [ ]:
def plot_conv(metrics, title='Conv'):
    # create a figure
    fig, ax1 = plt.subplots(figsize=(12,6))

    # loss
    ax1.set_xlabel('Epochs', fontsize=25)
    ax1.set_ylabel('Loss', fontsize=25)
    ax1.plot(metrics['train_loss'], label='Training Loss', linewidth=3)
    ax1.plot(metrics['eval_loss'], label='Test Loss', linestyle='dashed', linewidth=3)
    ax1.tick_params(axis='y', labelsize=20)
    ax1.tick_params(axis='x', labelsize=20)
    handles1, labels1 = ax1.get_legend_handles_labels()

    # accuracy
    ax2 = ax1.twinx() 
    ax2.set_ylabel('Accuracy', fontsize=25)
    ax2.plot(metrics['train_acc'], label='Train Acc', color='g', linewidth=3)
    ax2.plot(metrics['eval_acc'], label='Test Acc', color='r', linestyle='dashed', linewidth=3)
    ax2.tick_params(axis='y', labelsize=20)
    handles2, labels2 = ax2.get_legend_handles_labels()
    fig.tight_layout()

    plt.legend(handles1+handles2, labels1+labels2, fontsize=20, loc='center right')
    plt.title(title, fontsize=25)
    plt.show()

#### On Standard Data

Now, let's train a convolutional network directly on standard data.

In [ ]:
# define model
print('Preparing convolutional classifier... \n\n')
conv_standard = build_conv()
conv_standard = conv_standard.cuda()

set_seed(9999)
train_loader, test_loader = prepare_dataset()

# print model
summary(conv_standard, input_size=(1, 28, 28))

preprocess_standard = lambda x, y: (x.cuda(), y.cuda())
conv_standard, conv_standard_optim, conv_standard_metrics = train_conv(conv_standard, preprocess_standard)

In [ ]:
# save the model
save_model(conv_standard, conv_standard_optim, conv_standard_metrics, path='conv_standard.model')

# plot training curve
plot_conv(conv_standard_metrics, title='Standard')

#### On Flow Generated Data

Now, utilize normalizing flow as a data source and train a new classifier.

In [ ]:
# define model
print('Preparing 2nd convolutional classifier... \n\n')
conv_flow = build_conv()
conv_flow = conv_flow.cuda()

# print model
summary(conv_flow, input_size=(1, 28, 28))

# use flow-generated dataset instead of standard train set
def preprocess_flow(x, y, temp=0.75, clamp=True):
    # prepare y
    y = torch.randint(low=0, high=10, size=(256,))
    y = y.cuda()

    # prepare z and cond
    cond = F.one_hot(y, num_classes=10)
    z = temp * torch.cuda.FloatTensor(256, 1*28*28).normal_()

    # generate images
    with torch.no_grad():
        images, _ = flow(z, cond, rev=True)

    # clamp images
    if clamp:
        images = torch.clamp(images, 0., 1.)

    images = images.detach()
    images = images.view(-1, 1, 28, 28)
    return images, y

# train model on flow-generated dataset
conv_flow, conv_flow_optim, conv_flow_metrics = train_conv(conv_flow, preprocess_flow)

In [ ]:
# save the model
save_model(conv_flow, conv_flow_optim, conv_flow_metrics, path='conv_flow.model')

# plot training curve
plot_conv(conv_flow_metrics, title='Synthetic')

#### On Away Perturbed Flow-embeddings 

Now, use perturbed representations in Flow to generate images for a new classifier.

In [ ]:
# define model
print('Preparing 3nd convolutional classifier... \n\n')
conv_pert = build_conv()
conv_pert = conv_pert.cuda()

# print model
summary(conv_pert, input_size=(1, 28, 28))

# use away perturbed embeddings of flow
def preprocess_pert(x, y, pert=0.1):
    # preprocess for flow
    x = x.view(-1, 1*28*28)
    y_hot = F.one_hot(y, num_classes=10)

    # put data on cuda
    x, y_hot = x.cuda(), y_hot.cuda()

    with torch.no_grad():

        # embed images
        z, _ = flow(x, y_hot)

        # perturb images
        z = z + (1. + pert)

        # generate new images with flow
        images, _ = flow(z, y_hot, rev=True)

    images = images.detach()
    images = images.view(-1, 1, 28, 28)
    return images, y.cuda()

# train model on flow-perturbed samples
conv_pert, conv_pert_optim, conv_pert_metrics = train_conv(conv_pert, preprocess_pert)

In [ ]:
# save the model
save_model(conv_pert, conv_pert_optim, conv_pert_metrics, path='conv_pert.model')

# plot training curve
plot_conv(conv_pert_metrics, title='Away-LA')

#### On Randomized Flow-embeddings

Now, use randomized representations in Flow to generate images for a new classifier.

In [ ]:
# define model
print('Preparing 4th convolutional classifier... \n\n')
conv_rand = build_conv()
conv_rand = conv_rand.cuda()

set_seed(9999)
train_loader, test_loader = prepare_dataset()

# print model
summary(conv_rand, input_size=(1, 28, 28))

# use randomized embeddings of flow
def preprocess_rand(x, y, pert=0.15):
    # preprocess for flow
    x = x.view(-1, 1*28*28)
    y_hot = F.one_hot(y, num_classes=10)

    # put data on cuda
    x, y_hot = x.cuda(), y_hot.cuda()

    with torch.no_grad():

        # embed images
        z, _ = flow(x, y_hot)

        # perturb images
        z = z + pert * torch.cuda.FloatTensor(x.shape).normal_()

        # generate new images with flow
        images, _ = flow(z, y_hot, rev=True)

    images = images.detach()
    images = images.view(-1, 1, 28, 28)
    return images, y.cuda()

# train model on flow-perturbed samples
conv_rand, conv_rand_optim, conv_rand_metrics = train_conv(conv_rand, preprocess_rand)

In [ ]:
# save the model
save_model(conv_rand, conv_rand_optim, conv_rand_metrics, path='conv_rand.model')

# plot training curve
plot_conv(conv_rand_metrics, title='Randomized-LA')

#### On Adversarial Flow-embeddings

Now, use adversarial representations in Flow to generate images for a new classifier.

In [ ]:
# define model
print('Preparing 5th convolutional classifier... \n\n')
conv_adv = build_conv()
conv_adv = conv_adv.cuda()

set_seed(9999)
train_loader, test_loader = prepare_dataset()

# print model
summary(conv_adv, input_size=(1, 28, 28))

# use adversarial embeddings of flow
conv_loss = nn.CrossEntropyLoss()
conv_adv_pert = conv_adv
def preprocess_adv(x, y, pert=0.05, alpha=0.01, steps=10):
    flow.eval()
    conv_adv_pert.eval()

    # preprocess for flow
    x = x.view(-1, 1*28*28)
    y_hot = F.one_hot(y, num_classes=10)

    # put data on cuda
    x, y, y_hot = x.cuda(), y.cuda(), y_hot.cuda()

    with torch.no_grad():
        # embed images
        z, _ = flow(x, y_hot)

    dz = torch.zeros_like(z).to(z.device)
    dz.requires_grad = True
    for i in range(steps):
        with torch.enable_grad():
            # clean gradients
            flow.zero_grad()
            conv_adv_pert.zero_grad()

            #  feed-forward
            out, _ = flow(z + dz, y_hot, rev=True)
            out = out.view(-1, 1, 28, 28)

            preds = conv_adv_pert(out)
            loss = conv_loss(preds, y)

            # get gradient
            grad = torch.autograd.grad(
                loss, dz, retain_graph=False, create_graph=False
            )[0]

            dz = dz + torch.sign(grad) * alpha
            dz = torch.clamp(dz, -pert, +pert)

    # perturb images
    z = z + dz
    # generate new images with flow
    images, _ = flow(z, y_hot, rev=True)

    images = images.detach()
    images = images.view(-1, 1, 28, 28)
    return images, y.cuda()

# train model on flow-generated samples
conv_adv, conv_adv_optim, conv_adv_metrics = train_conv(conv_adv, preprocess_adv)

In [ ]:
# save the model
save_model(conv_adv, conv_adv_optim, conv_adv_metrics, path='conv_adv.model')

# plot training curve
plot_conv(conv_adv_metrics, title='Adversarial-LA')

#### On Image-Space Adversarial (PGD)

In [ ]:
# define model
print('Preparing 6th convolutional classifier... \n\n')
conv_pgd = build_conv()
conv_pgd = conv_pgd.cuda()

set_seed(9999)
train_loader, test_loader = prepare_dataset()

# print model
summary(conv_pgd, input_size=(1, 28, 28))

# use adversarial embeddings of flow
conv_loss = nn.CrossEntropyLoss()
conv_pgd_pert = conv_pgd
def preprocess_pgd(x, y, pert=0.3, alpha=0.01, steps=40):
    conv_pgd_pert.eval()

    # put data on cuda
    x, y = x.cuda(), y.cuda()

    dx = torch.zeros_like(x).to(x.device)
    dx.requires_grad = True
    for i in range(steps):
        with torch.enable_grad():
            # clean gradients
            conv_pgd_pert.zero_grad()

            preds = conv_pgd_pert(x + dx)
            loss = conv_loss(preds, y)

            # get gradient
            grad = torch.autograd.grad(
                loss, dx, retain_graph=False, create_graph=False
            )[0]

            dx = dx + torch.sign(grad) * alpha
            dx = torch.clamp(dx, -pert, +pert)

    # perturb images
    x = x + dx

    return x, y.cuda()

# train model on flow-generated samples
conv_pgd, conv_pgd_optim, conv_pgd_metrics = train_conv(conv_pgd, preprocess_pgd)

In [ ]:
# save the model
save_model(conv_pgd, conv_pgd_optim, conv_pgd_metrics, path='conv_pgd.model')

# plot training curve
plot_conv(conv_pgd_metrics, title='PGD')

## Evaluation

Reload models for evaluation purposes.

In [ ]:
# load state dictionaries
flow_dict = torch.load('flow.model')
conv_standard_dict = torch.load('conv_standard.model')
conv_flow_dict = torch.load('conv_flow.model')
conv_pert_dict = torch.load('conv_pert.model')
conv_rand_dict = torch.load('conv_rand.model')
conv_adv_dict = torch.load('conv_adv.model')

# load models
flow = build_flow()
flow = flow.cuda()
flow.load_state_dict(flow_dict['model_state_dict'])

conv_standard = build_conv()
conv_standard = conv_standard.cuda()
conv_standard.load_state_dict(conv_standard_dict['model_state_dict'])

conv_flow = build_conv()
conv_flow = conv_flow.cuda()
conv_flow.load_state_dict(conv_flow_dict['model_state_dict'])

conv_pert = build_conv()
conv_pert = conv_pert.cuda()
conv_pert.load_state_dict(conv_pert_dict['model_state_dict'])

conv_rand = build_conv()
conv_rand = conv_rand.cuda()
conv_rand.load_state_dict(conv_rand_dict['model_state_dict'])

conv_adv = build_conv()
conv_adv = conv_adv.cuda()
conv_adv.load_state_dict(conv_adv_dict['model_state_dict'])

conv_pgd = build_conv()
conv_pgd = conv_pgd.cuda()
conv_pgd.load_state_dict(conv_pgd_dict['model_state_dict'])

Now compare five classifier models in terms of accuracy.

In [ ]:
set_seed(9999)
_, test_loader = prepare_dataset()

def calc_acc(model, preprocess_fn):
    with torch.no_grad():
        accs = []
        for (x, y) in test_loader:
            # put data on cuda
            x, y = preprocess_fn(x, y)

            # feed-forward
            preds = model(x)
            preds = torch.argmax(preds, axis=1)

            # calculate accuracy
            acc = (preds == y) * 1.

            # save loss
            accs.append(torch.mean(acc))
        
        # calc mean validation loss
        acc = torch.mean(torch.Tensor(accs))
        print(f'Test Acc: {acc.cpu()}')

preprocess = preprocess_adv

conv_adv_pert = conv_standard
print('Running evalution on standard conv...')
calc_acc(conv_standard, preprocess)
print()

conv_adv_pert = conv_flow
print('Running evalution on synthetic conv...')
calc_acc(conv_flow, preprocess)
print()

conv_adv_pert = conv_pert
print('Running evalution on perturbed (away) Flow-embeddings conv...')
calc_acc(conv_pert, preprocess)
print()

conv_adv_pert = conv_rand
print('Running evalution on randomized Flow-embeddings conv...')
calc_acc(conv_rand, preprocess)
print()

conv_adv_pert = conv_adv
print('Running evalution on adversarial Flow-embeddings conv...')
calc_acc(conv_adv, preprocess)
print()

conv_adv_pert = conv_pgd
print('Running evalution on PGD conv...')
calc_acc(conv_pgd, preprocess)
print()

## Visualization

In [ ]:
set_seed(9999)
_, test_loader = prepare_dataset()

for images, labels in test_loader:
    images = images[:9, 0, ...]
    labels = labels[:9]
    break

def viz_inputs(preprocess_fn, images, labels):
    images, labels = preprocess_fn(images, labels)
    images, labels = images.cpu(), labels.cpu()

    if len(images.shape) == 4:
        images = images[:, 0, ...]

    f, axs = plt.subplots(3, 3, figsize=(12,12))
    for i in range(3):
        for j in range(3):
            index = i*3+j
            im = axs[i][j].imshow(images[index], cmap='gray')
            axs[i][j].set_title(f'Label: {labels[index]}', fontsize=25)
            axs[i][j].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
            #f.colorbar(im, ax=axs[i][j])
    return images

def preprocess(images, labels):
    return preprocess_adv(images, labels, pert=0.1, alpha=0.03, steps=10)

_ = viz_inputs(preprocess, images, labels)

In [ ]:
images_diff = np.abs(images_perturbed - images_clean)

f, axs = plt.subplots(3, 3, figsize=(12,12))
for i in range(3):
    for j in range(3):
        index = i*3+j
        im = axs[i][j].imshow(images_diff[index], cmap='BuPu')
        axs[i][j].set_title(f'Label: {labels[index]}', fontsize=25)
        axs[i][j].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
        #f.colorbar(im, ax=axs[i][j])